# LIBRARIES

In [1]:
import os
import ast
import math
import ccxt
import time
import pickle
import numpy as np
import configparser
import pandas as pd
import google.cloud
from pandas_gbq import gbq
import matplotlib.cm as cm
from boruta import BorutaPy
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
from google.cloud import bigquery
from sklearn import preprocessing
from IPython.display import clear_output
from sklearn.metrics import confusion_matrix
from scipy.interpolate import UnivariateSpline
from sklearn.ensemble import BaggingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

#_____

pd.set_option('display.max_rows', 5000)
pd.options.mode.chained_assignment = None

# FUNCTIONS

In [2]:
#FUNCIÓN QUE LEE EL ARCHIVO CONFIG
def get_config(category, key):
    
    global config
    return config[category][key]

#_____

#FUNCIÓN QUE GENERA CONEXIÓN CON ARCHIVO CONFIG
def updateConfig(config_name):
    
    global config
    
    config = configparser.ConfigParser()
    config.sections()
    config.read(config_name)
    
#_____
    
#REDONDEAR HACIA ABAJO
def round_decimals_down(number:float, decimals:int=2):
    """
    Returns a value rounded down to a specific number of decimal places.
    """
    if not isinstance(decimals, int):
        raise TypeError("decimal places must be an integer")
    elif decimals < 0:
        raise ValueError("decimal places has to be 0 or more")
    elif decimals == 0:
        return math.floor(number)

    factor = 10 ** decimals
    return math.floor(number * factor) / factor

#_____

#REDONDEAR HACIA ARRIBA
def round_decimals_up(number:float, decimals:int=2):
    """
    Returns a value rounded down to a specific number of decimal places.
    """
    if not isinstance(decimals, int):
        raise TypeError("decimal places must be an integer")
    elif decimals < 0:
        raise ValueError("decimal places has to be 0 or more")
    elif decimals == 0:
        return math.ceil(number)

    factor = 10 ** decimals
    return math.ceil(number * factor) / factor

#_____

#FUNCIÓN QUE ESCRIBE PICKLE DE VARIABLE
def writePickleVariable(variable,variable_name):
    pickle_out = open(variable_name+".pickle","wb")
    pickle.dump(variable, pickle_out)
    pickle_out.close()
    
#_____
    
#FUNCIÓN QUE LEE PICKLE DE VARIABLE
def readPickleVariable(variable_name):    
    return pickle.load(open(variable_name+".pickle","rb"))

#_____

#FUNCIÓN QUE DESCARGA BASE DE DATOS DE MERCADO DE BIGQUERY
def downloadDataBaseBigQuery(market_model,rows=None):
    
    #_____GOOGLE CLOUD CONECTION
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/ubuntu/SurfNet/predictionModels/bigQueryAccess.json"

    #_____SI NO SE DA UN NÚMERO ESPECÍFICO DE FILAS POR PARÁMETRO
    if rows==None:
    
        #_____DESCARGAR BASE DE DATOS
        candlesDataBase_BigQuery=gbq.read_gbq("SELECT * FROM [dogwood-terra-308100:surfNet."+market_model.replace("/","_")+"_1M] ORDER BY TIME DESC",project_id="dogwood-terra-308100",dialect="legacy").sort_values(by="TIME")
        candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)
        
    else:
        
        #_____DESCARGAR BASE DE DATOS
        candlesDataBase_BigQuery=gbq.read_gbq("SELECT * FROM [dogwood-terra-308100:surfNet."+market_model.replace("/","_")+"_1M] ORDER BY TIME DESC LIMIT "+str(rows),project_id="dogwood-terra-308100",dialect="legacy").sort_values(by="TIME")
        candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)

    candlesDataBase_BigQuery.OPEN=candlesDataBase_BigQuery.OPEN.astype(float)
    candlesDataBase_BigQuery.HIGH=candlesDataBase_BigQuery.HIGH.astype(float)
    candlesDataBase_BigQuery.LOW=candlesDataBase_BigQuery.LOW.astype(float)
    candlesDataBase_BigQuery.CLOSE=candlesDataBase_BigQuery.CLOSE.astype(float)
    candlesDataBase_BigQuery.VOLUME=candlesDataBase_BigQuery.VOLUME.astype(float)
    
    #_____REESTRUCTURAR BASE DE DATOS
    candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)
    
    #_____RETURN
    return candlesDataBase_BigQuery

#_____

#FUNCIÓN QUE DEVUELVE EL RSI DADO UN PERIODO DETERMINADO
def RSI(series, period):
    delta = series.diff().dropna()
    u = delta * 0
    d = u.copy()
    u[delta > 0] = delta[delta > 0]
    d[delta < 0] = -delta[delta < 0]
    u[u.index[period-1]] = np.mean( u[:period] ) #first value is sum of avg gains
    u = u.drop(u.index[:(period-1)])
    d[d.index[period-1]] = np.mean( d[:period] ) #first value is sum of avg losses
    d = d.drop(d.index[:(period-1)])
    rs = pd.DataFrame.ewm(u, com=period-1, adjust=False).mean() / \
         pd.DataFrame.ewm(d, com=period-1, adjust=False).mean()
    return 100 - 100 / (1 + rs)

#_____

#FUNCIÓN QUE CREA LAS VARIABLES SOBRE UNA BASE DE DATOS DADA
def variableCreation(candlesDataBase_BigQuery):
    
    global config
    
    #_____UPDATE CONFIG PREDICTION
    updateConfig("/home/ubuntu/SurfNet/predictionModels/ETH_FUT_1M/predictionModels_ETH_FUT_1M.ini")

    short_ma_hyper=int(get_config("PARAMETERS","SHORT_MA_HYPER"))
    short_ma_normal=int(get_config("PARAMETERS","SHORT_MA_NORMAL"))
    long_ma_normal=int(get_config("PARAMETERS","LONG_MA_NORMAL"))
    long_ma_hyper=int(get_config("PARAMETERS","LONG_MA_HYPER"))
    ultra_long_ma_normal=int(get_config("PARAMETERS","ULTRA_LONG_MA_NORMAL"))

    #_____VECTOR DE PRECIOS
    close_data_df=candlesDataBase_BigQuery.copy()
    close_data_df.CLOSE=close_data_df.CLOSE.astype(float)
    close_data_df=close_data_df[["CLOSE"]]

    #_____SUAVIZACIONES MA & EMA
    close_rolling_short_ma_hyper=close_data_df.rolling(short_ma_hyper).mean()
    close_rolling_short_ma_normal=close_data_df.rolling(short_ma_normal).mean()
    close_rolling_long_ma_normal=close_data_df.ewm(span=long_ma_normal).mean()
    close_rolling_long_ma_hyper=close_data_df.ewm(span=long_ma_hyper).mean()
    close_rolling_ultra_long_ma_normal=close_data_df.ewm(span=ultra_long_ma_normal).mean()

    #_____INSERTAR NUEVAS VARIABLES
    candlesDataBase_BigQuery["ROLLING_SHORT_MA_HYPER"]=close_rolling_short_ma_hyper
    candlesDataBase_BigQuery["ROLLING_SHORT_MA_NORMAL"]=close_rolling_short_ma_normal
    candlesDataBase_BigQuery["ROLLING_LONG_MA_NORMAL"]=close_rolling_long_ma_normal
    candlesDataBase_BigQuery["ROLLING_LONG_MA_HYPER"]=close_rolling_long_ma_hyper
    candlesDataBase_BigQuery["ROLLING_ULTRA_LONG_MA_NORMAL"]=close_rolling_ultra_long_ma_normal

    #_____ELIMINAR DATOS FALTANTES

    candlesDataBase_BigQuery.dropna(inplace=True)
    candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)
    
    #####
    
    #_____CÁLCULO DE LA PRIMERA DERIVADA
    x_CLOSE= np.arange(len(candlesDataBase_BigQuery))
    y_spl_CLOSE = UnivariateSpline(x_CLOSE,candlesDataBase_BigQuery[["CLOSE"]],s=0,k=3)
    Y_1D_CLOSE=y_spl_CLOSE.derivative(n=1)
    Y_1D_CLOSE=pd.DataFrame(Y_1D_CLOSE(x_CLOSE))
    candlesDataBase_BigQuery["Y_1D_CLOSE"]=Y_1D_CLOSE

    #_____CÁLCULO DE LA PRIMERA DERIVADA
    x_ROLLING_SHORT_MA_HYPER= np.arange(len(candlesDataBase_BigQuery))
    y_spl_ROLLING_SHORT_MA_HYPER = UnivariateSpline(x_ROLLING_SHORT_MA_HYPER,candlesDataBase_BigQuery[["ROLLING_SHORT_MA_HYPER"]],s=0,k=3)
    Y_1D_ROLLING_SHORT_MA_HYPER=y_spl_ROLLING_SHORT_MA_HYPER.derivative(n=1)
    Y_1D_ROLLING_SHORT_MA_HYPER=pd.DataFrame(Y_1D_ROLLING_SHORT_MA_HYPER(x_ROLLING_SHORT_MA_HYPER))
    candlesDataBase_BigQuery["Y_1D_ROLLING_SHORT_MA_HYPER"]=Y_1D_ROLLING_SHORT_MA_HYPER

    #_____CÁLCULO DE LA PRIMERA DERIVADA
    x_ROLLING_SHORT_MA_NORMAL= np.arange(len(candlesDataBase_BigQuery))
    y_spl_ROLLING_SHORT_MA_NORMAL = UnivariateSpline(x_ROLLING_SHORT_MA_NORMAL,candlesDataBase_BigQuery[["ROLLING_SHORT_MA_NORMAL"]],s=0,k=3)
    Y_1D_ROLLING_SHORT_MA_NORMAL=y_spl_ROLLING_SHORT_MA_NORMAL.derivative(n=1)
    Y_1D_ROLLING_SHORT_MA_NORMAL=pd.DataFrame(Y_1D_ROLLING_SHORT_MA_NORMAL(x_ROLLING_SHORT_MA_NORMAL))
    candlesDataBase_BigQuery["Y_1D_ROLLING_SHORT_MA_NORMAL"]=Y_1D_ROLLING_SHORT_MA_NORMAL

    #_____CÁLCULO DE LA PRIMERA DERIVADA
    x_ROLLING_LONG_MA_NORMAL= np.arange(len(candlesDataBase_BigQuery))
    y_spl_ROLLING_LONG_MA_NORMAL = UnivariateSpline(x_ROLLING_LONG_MA_NORMAL,candlesDataBase_BigQuery[["ROLLING_LONG_MA_NORMAL"]],s=0,k=3)
    Y_1D_ROLLING_LONG_MA_NORMAL=y_spl_ROLLING_LONG_MA_NORMAL.derivative(n=1)
    Y_1D_ROLLING_LONG_MA_NORMAL=pd.DataFrame(Y_1D_ROLLING_LONG_MA_NORMAL(x_ROLLING_LONG_MA_NORMAL))
    candlesDataBase_BigQuery["Y_1D_ROLLING_LONG_MA_NORMAL"]=Y_1D_ROLLING_LONG_MA_NORMAL

    #_____CÁLCULO DE LA PRIMERA DERIVADA
    x_ROLLING_LONG_MA_HYPER= np.arange(len(candlesDataBase_BigQuery))
    y_spl_ROLLING_LONG_MA_HYPER = UnivariateSpline(x_ROLLING_LONG_MA_HYPER,candlesDataBase_BigQuery[["ROLLING_LONG_MA_HYPER"]],s=0,k=3)
    Y_1D_ROLLING_LONG_MA_HYPER=y_spl_ROLLING_LONG_MA_HYPER.derivative(n=1)
    Y_1D_ROLLING_LONG_MA_HYPER=pd.DataFrame(Y_1D_ROLLING_LONG_MA_HYPER(x_ROLLING_LONG_MA_HYPER))
    candlesDataBase_BigQuery["Y_1D_ROLLING_LONG_MA_HYPER"]=Y_1D_ROLLING_LONG_MA_HYPER

    #_____CÁLCULO DE LA PRIMERA DERIVADA
    x_ROLLING_ULTRA_LONG_MA_NORMAL= np.arange(len(candlesDataBase_BigQuery))
    y_spl_ROLLING_ULTRA_LONG_MA_NORMAL = UnivariateSpline(x_ROLLING_ULTRA_LONG_MA_NORMAL,candlesDataBase_BigQuery[["ROLLING_ULTRA_LONG_MA_NORMAL"]],s=0,k=3)
    Y_1D_ROLLING_ULTRA_LONG_MA_NORMAL=y_spl_ROLLING_ULTRA_LONG_MA_NORMAL.derivative(n=1)
    Y_1D_ROLLING_ULTRA_LONG_MA_NORMAL=pd.DataFrame(Y_1D_ROLLING_ULTRA_LONG_MA_NORMAL(x_ROLLING_ULTRA_LONG_MA_NORMAL))
    candlesDataBase_BigQuery["Y_1D_ROLLING_ULTRA_LONG_MA_NORMAL"]=Y_1D_ROLLING_ULTRA_LONG_MA_NORMAL
    
    #####

    #_____CALCULAR VARIABLE RSI
    RSI_CLOSE = RSI(pd.Series(list(candlesDataBase_BigQuery.CLOSE.values)), int(get_config("PARAMETERS","RSI_LEN")))
    RSI_ROLLING_SHORT_MA_HYPER = RSI(pd.Series(list(candlesDataBase_BigQuery.ROLLING_SHORT_MA_HYPER.values)), int(get_config("PARAMETERS","RSI_LEN")))
    RSI_ROLLING_SHORT_MA_NORMAL = RSI(pd.Series(list(candlesDataBase_BigQuery.ROLLING_SHORT_MA_NORMAL.values)), int(get_config("PARAMETERS","RSI_LEN")))
    RSI_ROLLING_LONG_MA_NORMAL = RSI(pd.Series(list(candlesDataBase_BigQuery.ROLLING_LONG_MA_NORMAL.values)), int(get_config("PARAMETERS","RSI_LEN")))
    RSI_ROLLING_LONG_MA_HYPER = RSI(pd.Series(list(candlesDataBase_BigQuery.ROLLING_LONG_MA_HYPER.values)), int(get_config("PARAMETERS","RSI_LEN")))
    RSI_ROLLING_ULTRA_LONG_MA_NORMAL = RSI(pd.Series(list(candlesDataBase_BigQuery.ROLLING_ULTRA_LONG_MA_NORMAL.values)), int(get_config("PARAMETERS","RSI_LEN")))

    #_____AGRAGAR VARIABLE RSI A BASE DE DATOS
    candlesDataBase_BigQuery["RSI_CLOSE"]=RSI_CLOSE
    candlesDataBase_BigQuery["RSI_ROLLING_SHORT_MA_HYPER"]=RSI_ROLLING_SHORT_MA_HYPER
    candlesDataBase_BigQuery["RSI_ROLLING_SHORT_MA_NORMAL"]=RSI_ROLLING_SHORT_MA_NORMAL
    candlesDataBase_BigQuery["RSI_ROLLING_LONG_MA_NORMAL"]=RSI_ROLLING_LONG_MA_NORMAL
    candlesDataBase_BigQuery["RSI_ROLLING_LONG_MA_HYPER"]=RSI_ROLLING_LONG_MA_HYPER
    candlesDataBase_BigQuery["RSI_ROLLING_ULTRA_LONG_MA_NORMAL"]=RSI_ROLLING_ULTRA_LONG_MA_NORMAL
    
    #####

    #_____CREAR VARIABLE MACD
    macd_12 = close_data_df.ewm(span=int(get_config("PARAMETERS","MACD_SHORT")), adjust=False).mean()
    macd_26 = close_data_df.ewm(span=int(get_config("PARAMETERS","MACD_LONG")), adjust=False).mean()
    macd = macd_12 - macd_26
    macd_9 = macd.ewm(span=int(get_config("PARAMETERS","MACD_TRIGGER")), adjust=False).mean()

    #_____AGREGAR VARIABLES A LA BASE DE DATOS
    candlesDataBase_BigQuery["MACD"]=macd
    candlesDataBase_BigQuery["MACD_TRIGGER"]=macd_9
    
    #_____REESTRUCTURAR BASE DE DATOS
    candlesDataBase_BigQuery=candlesDataBase_BigQuery[int(get_config("PARAMETERS","ULTRA_LONG_MA_NORMAL"))+1:]
    candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)
    
    return candlesDataBase_BigQuery

#_____

#FUNCIÓN QUE ESCALA LAS VARIABLES CONTINUAS DE LA BASE DE DATOS
def scaleVariables(candlesDataBase_BigQuery):
    
    #_____ESCALAR VARIABLES
    continuous_variables=list(candlesDataBase_BigQuery.columns)

    scaler=preprocessing.StandardScaler(with_mean=True).fit(np.array(candlesDataBase_BigQuery[continuous_variables]))
    scaled_data=pd.DataFrame(scaler.transform(candlesDataBase_BigQuery[continuous_variables]))
    scaled_data.columns=continuous_variables

    for var in continuous_variables:
        candlesDataBase_BigQuery[str(var)] = scaled_data[str(var)]
    
    #_____RETURN
    return candlesDataBase_BigQuery

#_____

#FUNCIÓN QUE PREDICE SOBRE LOS DATOS TEST
def predictFunction(candlesDataBase_Actual,market_model):

    #_____DESCARGAR MODELO
    train_model=readPickleVariable("/home/ubuntu/SurfNet/predictionModels/MODELS/ETH_1M/TRAIN_MODEL_"+market_model.replace("/","_")+"_1M")
    
    #_____DATOS
    X_test=candlesDataBase_Actual.copy()
    X_test=X_test.values
    
    #_____PREDICTION
    prediction = train_model.predict(np.array(X_test))
    
    #_____RETURN
    return prediction

#_____

#FUNCIÓN QUE NOS DE AUTORIZACIÓN DE COMPRA EN MARKET MODEL
def autorizarCompra(market_model):
    
    #_____ACTUALIZAR ARCHIVO CONFIG
    updateConfig('frontTesting_ETH_FUT_1M_05K.ini')
    
    #_____CREAR BASE DE DATOS DE OBSERVACIÓN
    candlesDataBase_Binance = binance.fetch_ohlcv(market_model,"1m",limit=1000)
    candlesDataBase_Actual = pd.DataFrame(columns=["ID","TIME","MARKET","OPEN","HIGH","LOW","CLOSE","VOLUME"])

    for i in range(0,len(candlesDataBase_Binance)):
        candlesDataBase_Actual.at[i,"ID"]=candlesDataBase_Binance[i][0]
        candlesDataBase_Actual.at[i,"TIME"]=datetime.fromtimestamp(candlesDataBase_Binance[i][0]/1000.0)
        candlesDataBase_Actual.at[i,"MARKET"]=market_model
        candlesDataBase_Actual.at[i,"OPEN"]=candlesDataBase_Binance[i][1]
        candlesDataBase_Actual.at[i,"HIGH"]=candlesDataBase_Binance[i][2]
        candlesDataBase_Actual.at[i,"LOW"]=candlesDataBase_Binance[i][3]
        candlesDataBase_Actual.at[i,"CLOSE"]=candlesDataBase_Binance[i][4]
        candlesDataBase_Actual.at[i,"VOLUME"]=candlesDataBase_Binance[i][5]

    #_____DAR FORMATO A LAS COLUMNAS DE LA BASE DE DATOS CREADA
    candlesDataBase_Actual["ID"]=candlesDataBase_Actual["ID"].astype(str)
    candlesDataBase_Actual["TIME"]=pd.to_datetime(candlesDataBase_Actual["TIME"])
    candlesDataBase_Actual["MARKET"]=candlesDataBase_Actual["MARKET"].astype(str)
    candlesDataBase_Actual["OPEN"]=candlesDataBase_Actual["OPEN"].astype(float)
    candlesDataBase_Actual["HIGH"]=candlesDataBase_Actual["HIGH"].astype(float)
    candlesDataBase_Actual["LOW"]=candlesDataBase_Actual["LOW"].astype(float)
    candlesDataBase_Actual["CLOSE"]=candlesDataBase_Actual["CLOSE"].astype(float)
    
    #_____DESCARGAR 10.000 DATOS DE GOOGLE CLOUD
    candlesDataBase_BigQuery=downloadDataBaseBigQuery(market_model,rows=int(get_config("FRONTTESTING","BIGQUERY_ROWS")))

    #_____UNIR BASE DE DATOS BIGQUERY CON BASE DE DATOS ACTUAL + QUITAR DUPLICADOS
    candlesDataBase_BigQuery=candlesDataBase_BigQuery.append(candlesDataBase_Actual)
    candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)
    candlesDataBase_BigQuery.drop_duplicates(subset=["ID"],inplace=True)
    candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)

    #_____CREAR VARIABLES INPUT
    candlesDataBase_BigQuery=variableCreation(candlesDataBase_BigQuery)

    #_____SELECCIONAR VARIABLES NECESARIAS PARA CORRER EL MODELO
    variableList=readPickleVariable("/home/ubuntu/SurfNet/predictionModels/MODELS/ETH_1M/COLUMNS_"+market_model.replace("/","_")+"_1M")[3:-1]
    candlesDataBase_BigQuery=candlesDataBase_BigQuery[variableList]

    #_____ESCALAR VARIABLES SELECCIONADAS
    candlesDataBase_BigQuery=scaleVariables(candlesDataBase_BigQuery)

    #_____PREDECIR COMPRA O NO-COMPRA
    candlesDataBase_BigQuery["PRED"]=predictFunction(candlesDataBase_BigQuery,market_model)
    prediction_Actual=candlesDataBase_BigQuery.at[len(candlesDataBase_BigQuery)-1,"PRED"]
    
#     #_____SI LA PREDICCIÓN ME DICE QUE COMPRE
#     if prediction_Actual=="SI":
        
#         #_____CREAR BASE DE DATOS DE CLOSE PRICE
#         candles = binance.fetch_ohlcv(market_model, "1m",limit=1000)
#         close_data = []
#         for candle in candles:
#             close_data.append(candle[4])
#         close_data_df=pd.DataFrame(close_data,columns=["CLOSE"])

#         #_____HACER SUAVIZACIÓN NORMAL A CLOSE PRICE
#         last_minute_short=close_data_df.rolling(int(get_config("FRONTTESTING","ROLLING"))).mean()[-int(get_config("FRONTTESTING","TAIL")):]
#         last_minute_short.reset_index(inplace=True,drop=True)

#         #_____HALLAR PROPORCIÓN DE DATOS MENORES A VALOR ACTUAL
#         contador_close=0
#         for close in range(0,len(last_minute_short)):
#             if last_minute_short.at[len(last_minute_short)-1,"CLOSE"]>last_minute_short.at[close,"CLOSE"]:
#                 contador_close=contador_close+1

#         #_____SI LA OBSERVACIÓN ACTUAL MUESTRA UN CRECIMIENTO RESPECTO A LAS ANTERIORES OBSERVACIONES
#         if contador_close/len(last_minute_short) < float(get_config("FRONTTESTING","PROPORTION")):
#             prediction_Actual="NO"
#         else:
#             prediction_Actual="SI"
    
    return prediction_Actual

#_____

#FUNCIÓN QUE AUTORIZA VENTA
def autorizarVenta(market_model):
    
    global config
    global index_row
    global tradingBook
    global netSellPrice
    global netBuyAmount
    global markets_Utility
    global possibleUtility
    global sellAuthorization
    
    #_____ACTUALIZAR ARCHIVO CONFIG
    updateConfig('frontTesting_ETH_FUT_1M_05K.ini')

    #_____SIMULAR VENTA EN MERCADO SELECCIONADO CON CRYPTOS OBTENIDAS
    netSellPrice=simulateMarketSellCrypto(netBuyAmount,market_model)

    #_____CALCULAR UTILIDAD A OBTENER
    possibleUtility=round_decimals_down(netSellPrice/float(get_config("FRONTTESTING","BUDGET"))-1,6)

    #____IMPRIMIR RESULTADO PARCIAL
    tradingBook.at[index_row,"NET_SELL_PRICE"]=netSellPrice
    tradingBook.at[index_row,"NET_UTILITY"]=possibleUtility
    
    #____IMPRIMIR RESULTADO PARCIAL
    print(tradingBook[-1:])
    print("")
    print("[[UTILITY]]")
    print(markets_Utility)
    
    #_____SI DECIDO USAR O NO DATOS DE BACKTESTING
    if int(get_config("FRONTTESTING","BACKTESTING"))==0:
        UTILITY_THRESHOLD=float(get_config("FRONTTESTING","UTILITY_THRESHOLD"))
        STOPLOSS_THRESHOLD=float(get_config("FRONTTESTING","STOPLOSS_THRESHOLD"))
    else:
        UTILITY_THRESHOLD=readPickleVariable("/home/ubuntu/SurfNet/predictionModels/MODELS/ETH_1M/TAKEPROF_"+market_model.replace("/","_")+"_1M")
        STOPLOSS_THRESHOLD=readPickleVariable("/home/ubuntu/SurfNet/predictionModels/MODELS/ETH_1M/STOPLOSS_"+market_model.replace("/","_")+"_1M")

    #_____SI LA ALCANZA YA SEA EL TOPE ESTIPULADO O STOP-LOSS
    if (possibleUtility >= UTILITY_THRESHOLD) or (possibleUtility <= STOPLOSS_THRESHOLD):

        #_____ACTUALIZAR VARIABLE VENTA
        sellAuthorization=1

    #_____SI NO LA ALCANZA TOPES
    else:

        #_____ACTUALIZAR VARIABLE VENTA
        sellAuthorization=0
        
    #_____RETURN
    return sellAuthorization

#_____

#FUNCIÓN QUE SIMULA COMPRA DE CRYPTO CON FIAT/USDT
def simulateMarketBuyCrypto(tradingAmount,market_model):

    global config
    global binance
    
    #_____ACTUALIZAR ARCHIVO CONFIG
    updateConfig('frontTesting_ETH_FUT_1M_05K.ini')

    #_____VALIDAR QUE ORDERBOOK ALCANCE PARA HACER LA SIMULACIÓN
    while_break=0
    while while_break<tradingAmount*1.5:

        #_____DESCARGAR BUYINGBOOK
        buyingBook=pd.DataFrame(binance.fetch_order_book(market_model)["asks"],columns=["ASK_VALUE","ASK_SIZE"])
        buyingBook["ASK_PRICE"]=buyingBook["ASK_VALUE"]*buyingBook["ASK_SIZE"]

        #_____ACTUALIZAR SUMA DE VOLUMENES DE ORDENES EN ORDERBOOK DESCARGADO
        while_break=sum(buyingBook.ASK_PRICE.values)

        #_____SI VOLUMEN TOTAL ES MAYOR A LO BUSCADO, SEGUIR CON SIMULACIÓN -> DLC, PRINT + ESPERAR
        if while_break<tradingAmount*1.5:
            print("[ERROR]: simulateMarketBuyCrypto()")
            time.sleep(int(get_config("FRONTTESTING","TIME_SLEEP")))

    #_____SIMULACIÓN DE COMPRA CON USDT -> RETORNA VALOR DE CRYPTO ADQUIRIDO NETO
    usdt_used=0
    crypto_buyed=0
    usdt_used_acum=0
    crypto_buyed_acum=0
    contador_buying_book=0

    while usdt_used_acum<tradingAmount:

        usdt_used=buyingBook.at[contador_buying_book,"ASK_PRICE"]
        crypto_buyed=buyingBook.at[contador_buying_book,"ASK_SIZE"]

        if usdt_used_acum+usdt_used < tradingAmount:
            usdt_used_acum=usdt_used_acum+usdt_used
            crypto_buyed_acum=crypto_buyed_acum+crypto_buyed
            contador_buying_book=contador_buying_book+1
        else:
            diferencia=tradingAmount-usdt_used_acum
            usdt_used_acum=usdt_used_acum+diferencia

            porcentaje_diferencia=diferencia/usdt_used
            crypto_buyed_acum=crypto_buyed_acum+crypto_buyed*porcentaje_diferencia

    return crypto_buyed_acum*(1-float(get_config("FRONTTESTING","BUY_FEE")))

#_____

#FUNCIÓN QUE SIMULA ORDEN MARKET DE VENTA EN KRAKEN BTC-USDC
def simulateMarketSellCrypto(tradingAmount,market_model):
    
    global config
    global binance
    
    #_____ACTUALIZAR ARCHIVO CONFIG
    updateConfig('frontTesting_ETH_FUT_1M_05K.ini')

    #_____VARIABLES
    sellAmount=0.0
    sellCounter=0
    sellPriceACUM=0.0
    sellAmountACUM=0.0

    #_____VALIDAR QUE ORDERBOOK ALCANCE PARA HACER LA SIMULACIÓN
    while_break=0
    while while_break<tradingAmount*1.5:

        sellingBook=pd.DataFrame(binance.fetch_order_book(market_model)["bids"],columns=["BID_VALUE","BID_SIZE"])

        #ACTUALIZAR SUMA DE VOLUMENES DE ORDENES EN ORDERBOOK DESCARGADO
        while_break=sum(list(sellingBook.BID_SIZE.values))

        #SI VOLUMEN TOTAL ES MAYOR A LO BUSCADO, SEGUIR CON SIMULACIÓN -> DLC, PRINT + ESPERAR
        if while_break<tradingAmount*1.5:
            print("[ERROR]: simulateMarketSellCrypto()")
            time.sleep(int(get_config("FRONTTESTING","TIME_SLEEP")))

    #_____SIMULAR VENTA
    while sellAmountACUM<tradingAmount:
        sellAmount=float(sellingBook.iloc[sellCounter,1])
        sellAmountACUM=sellAmountACUM+sellAmount
        if sellAmountACUM<tradingAmount:
            sellPriceACUM=sellPriceACUM+(sellAmount*float(sellingBook.iloc[sellCounter,0]))
            sellCounter=sellCounter+1
        else:
            sellPriceACUM=sellPriceACUM+((tradingAmount-(sellAmountACUM-sellAmount))*float(sellingBook.iloc[sellCounter,0]))
            
    return sellPriceACUM*(1-float(get_config("FRONTTESTING","SELL_FEE")))

# PARAMETERS

In [3]:
#ACTUALIZAR ARCHIVO CONFIG
updateConfig('frontTesting_ETH_FUT_1M_05K.ini')

#BINANCE
binance = ccxt.binance()
binance.apiKey = get_config('FRONTTESTING', 'API_KEY')
binance.secret = get_config('FRONTTESTING', 'API_SECRET')

# CHECK

# START

In [4]:
#_____MARKET MODEL DE PRUEBA
market_models=ast.literal_eval(get_config("FRONTTESTING","MARKET_MODELS"))

#_____LOOP DE ITERACIÓN
for market_model in market_models:
    
    tradingBook=pd.DataFrame(columns=["DATE_BUY","DATE_SELL","MARKET","NET_BUY_PRICE","NET_SELL_PRICE","AMOUNT","NET_UTILITY"])
    writePickleVariable(tradingBook,"/home/ubuntu/SurfNet/predictionModels/RESULTS/ETH_1M/"+("TRADINGBOOK_"+market_model.replace("/","_")+"_1M_05K").lower())

# RUN

In [ ]:
#_____INICIALIZAR VARIABLES GLOBALES
netSellPrice=0
netBuyAmount=0
possibleUtility=0
sellAuthorization=0

#_____ACTUALIZAR ARCHIVO CONFIG
updateConfig('frontTesting_ETH_FUT_1M_05K.ini')

#_____MARKET MODEL DE PRUEBA
market_models=ast.literal_eval(get_config("FRONTTESTING","MARKET_MODELS"))

#_____UTILIDAD GLOBAL PARA IMPRESIÓN
markets_Utility=dict()
for key in market_models:
    markets_Utility[key]=0.0

#_____LOOP INFINITO
while True:
    
    #_____TRY DE CONTINGENCIA
    try:
    
        #_____LOOP DE ITERACIÓN
        for market_model in market_models:

            #_____LIMPIAR OUTPUT
            clear_output()

            #_____ACTUALIZAR ARCHIVO CONFIG
            updateConfig('frontTesting_ETH_FUT_1M_05K.ini')

            #_____DETERMINAR SI COMPRO O NO-COMPRO
            buyAuthorization=autorizarCompra(market_model)

            #_____PRINT
            print("MARKET: "+market_model+" , "+"BUY: "+buyAuthorization)
            print("[[UTILITY]]")
            print(markets_Utility)

            #_____ACTUALIZAR ARCHIVO CONFIG
            updateConfig('frontTesting_ETH_FUT_1M_05K.ini')

            #_____SI DECIDO COMPRAR
            if buyAuthorization=="SI":

                #_____SIMULAR COMPRA EN MERCADO SELECCIONADO CON BUDGET PROPUESTO
                netBuyAmount=simulateMarketBuyCrypto(float(get_config("FRONTTESTING","BUDGET")),market_model)

                #_____LEER BASE DE DATOS TRADING BOOK
                tradingBook=readPickleVariable("/home/ubuntu/SurfNet/predictionModels/RESULTS/ETH_1M/"+("TRADINGBOOK_"+market_model.replace("/","_")+"_1M_05K").lower())

                #_____ASIGNAR POSICIONAMIENTO DE FILAS
                index_row=len(tradingBook)

                #_____AGREGAR DATOS A BASE DE DATOS TRADE
                tradingBook.at[index_row,"DATE_BUY"]=(datetime.now()-timedelta(hours=5)).strftime("%d/%m/%y %H:%M:%S")
                tradingBook.at[index_row,"MARKET"]=market_model.replace("/","_")+"_1M"
                tradingBook.at[index_row,"NET_BUY_PRICE"]=float(get_config("FRONTTESTING","BUDGET"))
                tradingBook.at[index_row,"AMOUNT"]=netBuyAmount

                #_____ACTUALIZAR VARIABLE VENTA
                sellAuthorization=0

                #_____LOOP INFINITO DE VENTA
                while sellAuthorization==0:

                    #_____ESPERAR UN TIEMPO
                    time.sleep(int(get_config("FRONTTESTING","TIME_SLEEP")))

                    #_____LIMPIAR OUTPUT
                    clear_output()

                    #_____DETERMINAR SI VENDO O NO VENDO
                    sellAuthorization = autorizarVenta(market_model)

                #_____AGREGAR DATOS A BASE DE DATOS TRADE
                tradingBook.at[index_row,"DATE_SELL"]=(datetime.now()-timedelta(hours=5)).strftime("%d/%m/%y %H:%M:%S")
                tradingBook.at[index_row,"NET_SELL_PRICE"]=netSellPrice
                tradingBook.at[index_row,"NET_UTILITY"]=possibleUtility

                #_____EXPORTAR BASE DE DATOS DE COMPRA-VENTA
                writePickleVariable(tradingBook,"/home/ubuntu/SurfNet/predictionModels/RESULTS/ETH_1M/"+("TRADINGBOOK_"+market_model.replace("/","_")+"_1M_05K").lower())

                #_____ACTUALIZAR DICCIONARIO DE UTILIDADES
                markets_Utility[market_model]=sum(list(tradingBook.NET_UTILITY.values))

            #_____ESPERAR UN MOMENTO
            time.sleep(int(get_config("FRONTTESTING","TIME_SLEEP")))
        
    #_____EN CASO DE ERROR
    except:
        
        #_____ESPERAR UN RATO
        time.sleep(60)

In [ ]:
tradingBook.to_excel()